<a href="https://colab.research.google.com/github/rafabandoni/nfl-predict/blob/main/notebooks/03_nfl_predict_class_modeling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install optuna

In [ ]:
import pandas as pd
import seaborn as sns
import numpy as np

from abc import ABC, abstractmethod

from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_score
from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.preprocessing import StandardScaler
# from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier

import shap
import optuna
import pickle

# 0.3 Pre Processing

In [ ]:
featured_df = pd.read_parquet('https://github.com/rafabandoni/nfl-predict/raw/refs/heads/main/data/output/featured_df.parquet')
featured_df.head()

In [ ]:
test_df = pd.read_parquet('https://github.com/rafabandoni/nfl-predict/raw/refs/heads/main/data/output/test_data.parquet')
test_df.head()

## Creating preprocess pipeline

### Creating functions for pipeline

In [ ]:
# This function is outside of the class because it will be use inside the classes
def creating_home_and_away_columns(columns: list) -> list:
  new_named_list = []
  for item in columns:
    new_named_list.append(item + '_home_')
    new_named_list.append(item + '_away_')
  return new_named_list

In [ ]:
# Let's start by creating an abstract class
class BaseTransformer(BaseEstimator, TransformerMixin, ABC):
  def __init__(self):
    super().__init__()

  def fit(self, X, y=None):
    return self # in our case, there's nothing to fit here

  @abstractmethod
  def transform(self, X):
    pass

In [ ]:
# Cleaning % data
class CleanPercentData(BaseTransformer):
  def transform(self, X):
    X_transformed = X.copy()
    for column in X_transformed.columns:
      if '%' in column:
        X_transformed[column] = X_transformed[column] / 100
    return X_transformed

In [ ]:
# Remove special characters from columns
class RemoveSpecialCharColumns(BaseTransformer):
  def transform(self, X):
    X_transformed = X.copy()
    for column in X_transformed.columns:
      new_name = column.lower().replace(' ','_').replace('%', '_perc')
      X_transformed.rename(columns={
          column : new_name
      }, inplace=True)
    return X_transformed

In [ ]:
# Transform turnover in negative data
turnover_columns = [
    'defense_passing_lng',
    'defense_receiving_lng',
    'defense_rushing_lng',
    'defense_interceptions_lng',
    'offense_passing_lng',
    'offense_receiving_lng',
    'offense_rushing_lng',
    'special_teams_kickoff_returns_lng',
    'special_teams_punt_returns_lng',
    'special_teams_punting_lng'
]

class NegativeTurnoverNumbers(BaseTransformer):
  def __init__(self, columns):
    super().__init__()
    self.columns = columns

  def transform(self, X):
    X_transformed = X.copy()
    turnover_columns = creating_home_and_away_columns(self.columns)
    for column in turnover_columns:
      new_column = []
      for item in X_transformed[column]:
        if 'T' in item:
          new_value = item.replace('T', '')
          new_value = int(new_value)
          new_value = new_value * -1
        else:
          new_value = int(item)
        new_column.append(new_value)
      X_transformed[column] = new_column
    return X_transformed

In [ ]:
# Fix columns with A_M (attemps_made)
home_away_percent_columns_list = [
    'special_teams_field_goals_1-19_>_a-m',
    'special_teams_field_goals_20-29_>_a-m',
    'special_teams_field_goals_30-39_>_a-m',
    'special_teams_field_goals_40-49_>_a-m',
    'special_teams_field_goals_50-59_>_a-m',
    'special_teams_field_goals_60+_>_a-m'
]

# We will transform it in a percent so we don't need to create a new column for each case
class TransformHomeAwayPercentColumns(BaseTransformer):
  def __init__(self, columns):
    super().__init__()
    self.columns = columns

  def transform(self, X):
    X_transformed = X.copy()
    home_away_percent_columns_list = creating_home_and_away_columns(self.columns)
    for column in home_away_percent_columns_list:
      attps_list = X_transformed[column].str.split('_').str[0]
      matches_list = X_transformed[column].str.split('_').str[1]

      percent_matches = matches_list.astype('int') / attps_list.astype('int')
      percent_matches.fillna(0, inplace=True)
      X_transformed[column] = percent_matches
      X_transformed.rename(columns={
          column : column.replace('a-m', 'percent_a_m')
      }, inplace=True)
    return X_transformed

In [ ]:
# Removing non important columns (trash from the origin)
columns_trash = ['data_fgm',
                   'data_fg__perc',
                   'data_xpm',
                   'data_xp_pct',
                   'data_kret_td',
                   'data_pret_t']

weather_columns = ['weather_temperature',
                   'weather_wind_mph',
                   'weather_humidity',
                   'weather_detail']

class DropUnusedColumns(BaseTransformer):
  def __init__(self, columns_trash, weather_columns):
    super().__init__()
    self.columns_trash = columns_trash
    self.weather_columns = weather_columns

  def transform(self, X):
    X_transformed = X.copy()
    columns_trash = creating_home_and_away_columns(self.columns_trash)
    X_transformed.drop(columns_trash,
                       axis=1,
                       inplace=True)
    X_transformed.drop(self.weather_columns,
                       axis=1,
                       inplace=True)
    return X_transformed

In [ ]:
class FixDataType(BaseTransformer):
  def __init__(self, columns, data_type):
    super().__init__()
    self.columns = columns
    self.data_type = data_type

  def transform(self, X):
    X_transformed = X.copy()
    for column in self.columns:
      X_transformed[column] = X_transformed[column].astype(self.data_type)
    return X_transformed

In [ ]:
class CreateHomeWinner(BaseTransformer):
  def home_winner(self, score_home, score_away):
    if score_home > score_away:
      return True
    else:
      return False # we will not be working on ties here since they are rare on NFL

  def transform(self, X):
    X_transformed = X.copy()
    X_transformed['home_winner'] = X_transformed.apply(
        lambda row: self.home_winner(row['score_home'], row['score_away']),
        axis=1
    )
    return X_transformed

### Creating pipeline

In [ ]:
pipeline_steps = [ # TODO: add feature engineering transformations
    ('clean_percent_data', CleanPercentData()),
    ('remove_special_char_columns', RemoveSpecialCharColumns()),
    ('negative_turnover_number', NegativeTurnoverNumbers(columns=turnover_columns)),
    ('transform_home_away_columns_percent', TransformHomeAwayPercentColumns(columns=home_away_percent_columns_list)),
    ('drop_columns', DropUnusedColumns(columns_trash=columns_trash, weather_columns=weather_columns)),
    ('fix_datatype', FixDataType(columns=['schedule_date'], data_type='datetime64[ns]')),
    ('home_winner', CreateHomeWinner()),
]

In [ ]:
pipe = Pipeline(
    pipeline_steps
)

### Applying pipeline

In [ ]:
pipe.fit(test_df)
test_df_transformed = pipe.transform(test_df)

In [ ]:
test_df_transformed

## Variables defining - to update below

**IMPORTANT**: X, y and train test split before making preprocessing to avoid data leakage

In [ ]:
target = 'home_winner'
X_train = featured_df.drop(target, axis=1)
y_train = featured_df[[target]]
X_test = test_df.drop(target, axis=1)
y_test = test_df[[target]]

In [ ]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 0.3.1 ML Model

## Building model

In [ ]:
#Define the objective function for Optuna
def objective(trial):
    params = {
        'n_estimators': trial.suggest_int('n_estimators', 200, 800),
        'max_depth': trial.suggest_int('max_depth', 2, 25),
        'min_child_weight': trial.suggest_int('min_child_weight', 2, 20),
        'learning_rate': trial.suggest_float('learning_rate', 0.01, 0.3),
        'subsample': trial.suggest_float('subsample', 0.2, 0.8),
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'gamma': trial.suggest_float('gamma', 1e-8, 2, log=True),
        'reg_lambda': trial.suggest_float('reg_lambda', 1e-3, 10.0),
        'alpha': trial.suggest_float('alpha', 1, 15.0, log=True),
        'scale_pos_weight': trial.suggest_float('scale_pos_weight', 0.1, 5.0)
    }

    model = XGBClassifier(
        **params,
        # use_label_encoder=False,
        eval_metric='mlogloss',
        # enable_categorical=True,
        tree_method='hist',
        device= 'cuda',
        # objective='multi:softmax',
        objective='binary:logistic',
        # num_class=2
    )
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    # mcc = matthews_corrcoef(y_test, y_pred)
    # acc = accuracy_score(y_test, y_pred)
    skf = StratifiedKFold(n_splits=5, shuffle=True)
    acc = cross_val_score(model, X_train, y_train, cv=skf, scoring='accuracy').mean()
    trial.set_user_attr("acc", acc)
    return acc

# Callback to print the MCC score for each trial
# def print_mcc_callback(study, trial):
#     mcc = trial.user_attrs["mcc"]
#     print(f"Trial {trial.number}: MCC = {mcc}")

# def print_accuracy_callback(study, trial):
#     acc = trial.user_attrs["acc"]
#     print(f"Trial {trial.number}: Accuracy score = {acc}")

# Optimize hyperparameters with Optuna
study = optuna.create_study(direction='maximize')
study.optimize(objective, n_trials=30)

# Get the best parameters
best_params = study.best_params
print(f"Best parameters: {best_params}")

In [ ]:
xgb = XGBClassifier(
    **best_params,
    eval_metric='mlogloss',
    tree_method='hist',
    device= 'cuda',
    objective='binary:logistic'
)
xgb.fit(X_train, y_train)

## Predicting

In [ ]:
predictions = xgb.predict(X_test)

## Evaluating model

In [ ]:
cf_matrix = confusion_matrix(y_test, predictions)

group_names = ['True Neg','False Pos','False Neg','True Pos']
group_counts = ['{0:0.0f}'.format(value) for value in
                cf_matrix.flatten()]
group_percentages = ['{0:.2%}'.format(value) for value in
                     cf_matrix.flatten()/np.sum(cf_matrix)]
labels = [f'{v1}\n{v2}\n{v3}' for v1, v2, v3 in
          zip(group_names,group_counts,group_percentages)]
labels = np.asarray(labels).reshape(2,2)
sns.heatmap(cf_matrix, annot=labels, fmt='', cmap='Reds')

In [ ]:
print(classification_report(y_test, predictions))

In [ ]:
# explainer = shap.Explainer(xgb, X_test)
explainer = shap.Explainer(xgb, pd.DataFrame(X_train.astype('float64'), columns=X.columns))

# shap_values = explainer(X_test)
shap_values = explainer(pd.DataFrame(X_test, columns=X.columns))

shap.plots.waterfall(shap_values[0])

In [ ]:
shap.plots.beeswarm(shap_values)

## Saving model

In [ ]:
# save
with open('model_nfl.pkl','wb') as f:
    pickle.dump(xgb,f)